## requirements

In [ ]:
!apt-get update
!apt-get install g++ openjdk-8-jdk python-dev python3-dev
!pip3 install JPype1-py3 konlpy pandas tqdm
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
!JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"

## read data

In [ ]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/fllff/nlp_basic_lec/master/ratings_train.txt',sep='\t')

# NaN 삭제
df = df.dropna()

print(df)

## tokenization

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()

mecab.morphs('한국어 토큰 분리기 사용방법')


In [ ]:
from tqdm import tqdm

for i, row in tqdm(df.iterrows()):
    df.at[i,'document'] = mecab.morphs(row['document'])
    
print(df)                                

In [ ]:
doc_list = df['document']
label_list = df['label']

doc_len_list = [len(doc) for doc in doc_list]

# 문장길이 분포 확인
distribution = [0]*20
for doc_len in doc_len_list:
    distribution[int(doc_len/10)] += 1
print(distribution)

# label균형 확인
label_list.value_counts()

In [ ]:
# 문장 길이 trim
doc_list = [doc[:50] for doc in doc_list]

# 문장길이 분포 확인
distribution = [0]*20
for doc in doc_list:
    distribution[int(len(doc)/10)] += 1
print(distribution)

## vectorization


In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(doc_list, size=16, window=5, min_count=50, workers=2, iter=3)


In [ ]:
model.wv.most_similar('배우')

In [ ]:
model.wv['배우']

In [ ]:
# token -> vector 변환
vector_list = []
for doc in tqdm(doc_list):
    tmp = []
    for token in doc:
        try:
            tmp.append(model.wv[token].tolist())
        except:
            tmp.append([0]*10)
        
    vector_list.append(tmp)
    
print(vector_list[0])


In [ ]:
MAX_LEN = 50
tmp=[]

for i, vector in tqdm(enumerate(vector_list)):
    vector_list[i].extend([[0]*16 for x in range(MAX_LEN-len(vector))])

print(len(vector_list[0]))

## Model

In [ ]:
import tensorflow as tf
tf.__version__